In [1]:
import tensorflow as tf
import os
#定义神经网络的相关参数
INPUT_NODE=784
OUTPUT_NODE=10
LAYER1_NODE=500

#通过tf.get_varibale()函数来获取变量

def get_weight_variable(shape, regularizer):
    weights = tf.get_variable("weights", shape, initializer=tf.truncated_normal_initializer(stddev=0.1))
    if(regularizer != None):
        tf.add_to_collection('losses', regularizer(weights))
    return weights

#定义神经网络的前向传播过程
def inference(input_tensor, regularizer):
    with tf.variable_scope('layer1'):
        weights = get_weight_variable([INPUT_NODE, LAYER1_NODE], regularizer)
        biases = tf.get_variable("biases", [LAYER1_NODE], initializer=tf.constant_initializer(0.0))
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights) + biases)

    with tf.variable_scope('layer2'):
        weights = get_weight_variable([LAYER1_NODE, OUTPUT_NODE], regularizer)
        biases = tf.get_variable("biases", [OUTPUT_NODE], initializer=tf.constant_initializer(0.0))
        layer2 = tf.matmul(layer1, weights) + biases

    return layer2


In [2]:
#训练程序
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

#定义神经网络结构的参数
BATCH_SIZE=128
LEARNING_RATE_BASE=0.8
LEARNING_RATE_DECAY=0.99
REGULARIZATION_RATE=0.0001
TRAINING_STEPS=2001
MOVING_AVERAGE_DECAY=0.99
MODEL_SAVE_PTH='./aa/'
MODE_NAME='mnist_model'
def train(mnist):
    x=tf.placeholder(tf.float32,[None,INPUT_NODE],name='X')
    y_=tf.placeholder(tf.float32,[None,OUTPUT_NODE],name='Y')
    
    regularizer=tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    y=inference(x,regularizer)
    global_step=tf.Variable(0,trainable=False)
    
    #定义损失函数、学习率、滑动平均值以及训练过程
    variable_average=tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
    variable_averages_op=variable_average.apply(tf.trainable_variables())
    cross_entropy=tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,labels=tf.argmax(y_,1))
    cross_entropy_mean=tf.reduce_mean(cross_entropy)
    loss=cross_entropy_mean+tf.add_n(tf.get_collection('losses'))
    learning_rate=tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples/BATCH_SIZE,
        LEARNING_RATE_DECAY,
        staircase=True
    )
    train_step=tf.train.AdamOptimizer(learning_rate).minimize(loss,global_step)
    with tf.control_dependencies([train_step,variable_averages_op]):
        train_op=tf.no_op(name='train')
    
    #初始化Tensorflow持久化类
    saver=tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        for i in range(TRAINING_STEPS):
            xs,ys=mnist.train.next_batch(BATCH_SIZE)
            _,loss_value,step=sess.run([train_op,loss,global_step],feed_dict={x:xs,y_:ys})
            if(i%100==0):
                print("After %d training step(s),loss on training batch is %g."%(step,loss_value))
                saver.save(sess,os.path.join(MODEL_SAVE_PTH,MODE_NAME),global_step)
                

In [3]:
def main():
    mnist=input_data.read_data_sets('/tmp/data/',one_hot=True)
    train(mnist)
if __name__=='__main__':
    main()

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
After 1 training step(s),loss on training batch is 2.80992.
After 101 training step(s),loss on training batch is 88.9508.
After 201 training step(s),loss on training batch is 43.4766.
After 301 training step(s),loss on training batch is 23.6836.
After 401 training step(s),loss on training batch is 16.5619.
After 501 training step(s),loss on training batch is 10.4843.
After 601 training step(s),loss on training batch is 7.76211.
After 701 training step(s),loss on training batch is 5.73705.
After 801 training step(s),loss on training batch is 4.70326.
After 901 training step(s),loss on training batch is 3.90723.
After 1001 training step(s),loss on training batch is 3.94096.
After 1101 training step(s),loss on training batch is 3.63851.
After 1201 training step(s),loss on training batch is 3.45963.
Aft